# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# Libraries

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [2]:
import pandas as pd
admission=pd.read_csv("C:/Users/milena.xavier/Desktop/Iron/Labs/week2/lab-advanced-pandas/your-code/data/Admission_Predict.csv")
admission.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Remove trailing spaces at the end of the column names if there are any.


In [3]:
admission.columns = admission.columns.str.rstrip()

Let's evaluate the dataset by looking at the `head` function.

In [4]:
admission.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [5]:
admission.isna().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [6]:
admission.set_index('Serial No.')

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84
383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [7]:
admission['check']=admission['GRE Score'].astype(str)  + admission['CGPA'].astype(str) 
                    
print(admission['check'].nunique())
print(admission['check'].count())

duplicateRows = admission[admission.duplicated(['check'])]
duplicateRows
#there is no duplicates

385
385


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,check


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [8]:
admission.reset_index(inplace=True)
admission.head()

,index,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,check
0,0,1,337,118,4,4.5,4.5,9.65,1,0.92,3379.65
1,1,2,316,104,3,3.0,3.5,8.00,1,0.72,3168.0
2,2,3,322,110,3,3.5,2.5,8.67,1,0.80,3228.67
3,3,4,314,103,2,2.0,3.0,8.21,0,0.65,3148.21
4,4,5,330,115,5,4.5,3.0,9.34,1,0.90,3309.34


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [9]:
len(admission.loc[(admission['CGPA']>9) & (admission['Research']==1)])

101

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [10]:
display(admission.loc[(admission['CGPA']>9) & (admission['SOP']<3.5)])
(admission.loc[(admission['CGPA']>9) & (admission['SOP']<3.5)])["Chance of Admit"].mean()

,index,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,check
28,28,29,338,118,4,3.0,4.5,9.40,1,0.91,3389.4
62,62,63,327,114,3,3.0,3.0,9.02,0,0.61,3279.02
140,140,141,326,114,3,3.0,3.0,9.11,1,0.83,3269.11
217,217,218,324,111,4,3.0,3.0,9.01,1,0.82,3249.01
381,381,382,325,107,3,3.0,3.5,9.11,1,0.84,3259.11


0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [11]:
import numpy as np

In [12]:
def standardize(col):
    return col / np.linalg.norm(col)
  
    
    


Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [17]:
admission['CGPA_std']=standardize(admission['CGPA'])
admission['GRE_std']=standardize(admission['GRE Score'])
admission['LOR_std']=standardize(admission['LOR'])
admission.head()




,index,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,check,CGPA_std,GRE_std,LOR_std
0,0,1,337,118,4,4.5,4.5,9.65,1,0.92,3379.65,0.057067,0.054190,0.064673
1,1,2,316,104,3,3.0,3.5,8.00,1,0.72,3168.0,0.047309,0.050813,0.050301
2,2,3,322,110,3,3.5,2.5,8.67,1,0.80,3228.67,0.051272,0.051778,0.035929
3,3,4,314,103,2,2.0,3.0,8.21,0,0.65,3148.21,0.048551,0.050492,0.043115
4,4,5,330,115,5,4.5,3.0,9.34,1,0.90,3309.34,0.055234,0.053064,0.043115


We will generate the decision choice at random using the code below. Please run the cell.

In [18]:
# Libraries
from random import choices

In [32]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admission.shape[0])
decision_choice


['LOR_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'LOR_std'

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [34]:
deciding_column = admission.lookup(admission.index, decision_choice)
admission['deciding_column'] = deciding_column
admission.head()

C:\Users\milena.xavier\AppData\Local\Temp\ipykernel_13816\2951025286.py:1: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  deciding_column = admission.lookup(admission.index, decision_choice)


,index,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,check,CGPA_std,GRE_std,LOR_std,deciding_column
0,0,1,337,118,4,4.5,4.5,9.65,1,0.92,3379.65,0.057067,0.054190,0.064673,0.064673
1,1,2,316,104,3,3.0,3.5,8.00,1,0.72,3168.0,0.047309,0.050813,0.050301,0.050813
2,2,3,322,110,3,3.5,2.5,8.67,1,0.80,3228.67,0.051272,0.051778,0.035929,0.051272
3,3,4,314,103,2,2.0,3.0,8.21,0,0.65,3148.21,0.048551,0.050492,0.043115,0.050492
4,4,5,330,115,5,4.5,3.0,9.34,1,0.90,3309.34,0.055234,0.053064,0.043115,0.053064


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [36]:
def decision(x):
    if x > 0.8:
        return 1
    else:
        return 0
    
admission['decision']=admission["deciding_column"].apply(decision)


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [39]:
admission['decision'].value_counts()
#all students will be reproved :((

0    385
Name: decision, dtype: int64

In [40]:
display(admission['deciding_column'].max())

0.07185881253804778

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [55]:
admission.columns = admission.columns.str.replace(' ', '_').str.replace('[^a-zA-Z]', '').str.lower()

C:\Users\milena.xavier\AppData\Local\Temp\ipykernel_13816\422899707.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  admission.columns = admission.columns.str.replace(' ', '_').str.replace('[^a-zA-Z]', '').str.lower()


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [65]:
condition=[admission['universityrating']>4, admission['universityrating']<=4]
values=[admission['grescore']+10, admission['grescore']]
admission['adjusted_gre']=np.select(condition,values)


labels1 = ['Very Low', 'Low', 'High', 'Very High']
bins = pd.cut(admission['adjusted_gre'],4, labels=labels1)
bins

0      Very High
1            Low
2           High
3            Low
4      Very High
         ...    
380         High
381         High
382         High
383          Low
384         High
Name: adjusted_gre, Length: 385, dtype: category
Categories (4, object): ['Very Low' < 'Low' < 'High' < 'Very High']